In [345]:
import numpy as np
import pandas as pd
import random
import math

df = pd.read_csv("HousingPrice.csv")

Price = df['price']
FloorArea = df['lotsize']
NoOfBedrooms = df['bedrooms']
NoOfBathrooms = df['bathrms']

## Feature scaling on FlooeArea

In [346]:
FloorAreaMean = np.mean(FloorArea)
FloorAreaMin = min(FloorArea)
FloorAreaMax = max(FloorArea)
FloorAreaScaled = (FloorArea - FloorAreaMean)/(FloorAreaMax - FloorAreaMin)
FloorArea, FloorAreaScaled

(0      5850
 1      4000
 2      3060
 3      6650
 4      6360
        ... 
 541    4800
 542    6000
 543    6000
 544    6000
 545    6000
 Name: lotsize, Length: 546, dtype: int64,
 0      0.048092
 1     -0.079056
 2     -0.143661
 3      0.103075
 4      0.083143
          ...   
 541   -0.024073
 542    0.058401
 543    0.058401
 544    0.058401
 545    0.058401
 Name: lotsize, Length: 546, dtype: float64)

## Creating useful variables

In [347]:
n, m = df.shape
n, m

(546, 13)

In [348]:
n_train = math.floor(n * 0.70)
n_test = n - n_train
n_train, n_test

(382, 164)

In [349]:
col = [1] * n
data_scaling = list(zip(col,FloorAreaScaled,NoOfBedrooms,NoOfBathrooms))
len(data_scaling)

546

In [350]:
# X_train, Y_train, X_test, Y_test with scaling
X_train = np.array(data_scaling[:n_train])
Y_train = np.array(Price[:n_train])

X_test = np.array(data_scaling[n_train+1:])
Y_test = np.array(Price[n_train+1:])

In [351]:
X_train.shape

(382, 4)

## Function to find slope

In [352]:
def Slope(theta, X_train, Y_train):
    slope = np.zeros(theta.shape)
    for i in range(len(X_train)):
        itr = 0
        for j in range(theta.shape[1]):
            itr = itr + theta[0][j] * X_train[i][j]
        slope += (itr - Y_train[i]) * X_train[i]
    return slope

# Using batch gradient without regularisation

In [353]:
lr = 0.001
theta = np.zeros((1,4))
theta

array([[0., 0., 0., 0.]])

In [354]:
for i in range(5000):
    theta = theta - (lr/n_train)*Slope(theta,X_train,Y_train)
theta

array([[ 7645.16561033,  7527.93232483, 10966.99872425, 18651.19085438]])

In [355]:
theta = theta.reshape((4,1))
theta.shape

(4, 1)

In [356]:
Y_pred = np.dot(X_test, theta)
error = 0
for i in range(n_test-1):
    error += abs((Y_test[i] - Y_pred[i][0])/Y_test[i])
error /= n_test
print("MEAN ABSOLUTE PERCENTAGE ERROR: ")
float(error*100)

MEAN ABSOLUTE PERCENTAGE ERROR: 


20.342942821184607

# Using batch gradient with regularisation

In [357]:
lr = 0.001
Lambda = -120
theta = np.zeros((1,4))
theta

array([[0., 0., 0., 0.]])

In [358]:
for epochs in range(5000):
    cof = 1 - ((lr*Lambda)/n_train)
    theta = cof*theta - (lr/n_train)*Slope(theta, X_train,Y_train)
theta

array([[10854.7790114 , 18034.6716255 ,  5068.24913908, 31438.87102068]])

In [359]:
theta = theta.reshape((4,1))
theta.shape

(4, 1)

In [360]:
theta

array([[10854.7790114 ],
       [18034.6716255 ],
       [ 5068.24913908],
       [31438.87102068]])

In [361]:
Y_pred = np.dot(X_test, theta)
error = 0
for i in range(n_test-1):
    error += abs((Y_test[i] - Y_pred[i])/Y_test[i])
error = error/n_test

print("MEAN ABSOLUTE PERCENTAGE ERROR: ")
float(error*100)

MEAN ABSOLUTE PERCENTAGE ERROR: 


18.92124816522627

# Using Stochastic gradient without regularisation.

In [362]:
# FUNCTION TO FIND SLOPE FOR STOCHASTIC DESCENT
def SlopeStoch(theta, X_train, Y, ind):
	itr = 0
	for j in range(theta.shape[1]):
		itr = itr + theta[0][j]*X_train[j]
	return (itr - Y) * X_train[ind]


In [371]:
lr = 0.00001
theta = np.zeros((1,4))
theta

array([[0., 0., 0., 0.]])

In [372]:
for iter in range(10):
    for i in range(X_train.shape[0]):
        theta = theta - (lr)*SlopeStoch(theta, X_train, Y_train[i], i)
theta

array([[2222.39526393,   16.94489919, 5546.26936474, 3125.24588452]])

In [373]:
theta = theta.reshape((4,1))
theta.shape

(4, 1)

In [374]:
Y_pred = np.dot(X_test, theta)
error = 0
for i in range(n_test-1):
    error += abs((Y_test[i] - Y_pred[i])) / Y_test[i]
error = error/n_test

print("MEAN ABSOLUTE PERCENTAGE ERROR: ")
float(error*100)

MEAN ABSOLUTE PERCENTAGE ERROR: 


68.20015056290227

# Using Stochastic gradient with Regularization

In [375]:
lr = 0.00001
Lambda = 30
theta = np.zeros((1,4))
theta

array([[0., 0., 0., 0.]])

In [376]:
for itr in range(10):
  for i in range(X_train.shape[0]):
    cof = 1-((Lambda*lr)/n_train)
    theta = cof*theta - lr*(SlopeStoch(theta, X_train, Y_train[i], i))
theta

array([[2219.19847494,   16.93408054, 5538.96715982, 3120.76286454]])

In [377]:
theta = theta.reshape((4,1))
theta.shape

(4, 1)

In [378]:
Y_pred = np.dot(X_test, theta)
error = 0
for i in range(n_test-1):
    error += abs((Y_test[i] - Y_pred[i])) / Y_test[i]
error = error/n_test

print("MEAN ABSOLUTE PERCENTAGE ERROR: ")
float(error*100)

MEAN ABSOLUTE PERCENTAGE ERROR: 


68.24222400621994

# Using Minibatch gradient without regularisation

In [379]:
# USEFUL FUNCTIONS FOR MINI BATCH GRADIENT 
def product(w,x):
    ans=0
    for i in range(x.shape[0]):
        ans= ans + w[0][i]*x[i]
    return ans

def gradient(x,w,y,st,bs):
    ans=np.zeros(w.shape)
    for i in range(st,st+bs):
        if(i>=x.shape[0]):
          break
        ans=(ans+(product(w,x[i])-y[i])*x[i])
    return ans


In [380]:
BatchSize = 20;
lr = 0.0002
theta = np.zeros((1,4))
theta

array([[0., 0., 0., 0.]])

In [381]:
for itr in range(50):
  for i in range(BatchSize):
    theta = theta - (lr*gradient(X_train,theta,Y_train,BatchSize*i,BatchSize))/BatchSize
theta

array([[ 4891.71936057,   221.38545923, 14598.66776056,  7156.34800691]])

In [382]:
theta = theta.reshape((4,1))
theta.shape

(4, 1)

In [383]:
Y_pred = np.dot(X_test, theta)
error = 0
for i in range(n_test-1):
    error += abs((Y_test[i] - Y_pred[i])/Y_test[i])
error = error/n_test

print("MEAN ABSOLUTE PERCENTAGE ERROR: ")
float(error*100)

MEAN ABSOLUTE PERCENTAGE ERROR: 


25.794843431025104

# Using Minibatch gradient with regularisation for batch size = 20

In [385]:
BatchSize = 20;
lr = 0.0002
Lambda = -20
theta = np.zeros((1,4))
theta

array([[0., 0., 0., 0.]])

In [386]:
for itr in range(50):
  for i in range(BatchSize):
    temp = 1-((Lambda*lr)/BatchSize)
    theta = (temp * theta) - (lr * gradient(X_train, theta, Y_train, BatchSize*i, BatchSize))/ BatchSize

theta

array([[ 5231.20289362,   247.60936576, 15585.22770405,  7672.11402056]])

In [387]:
theta = theta.reshape((4,1))
theta.shape

(4, 1)

In [388]:
Y_pred = np.dot(X_test, theta)
error = 0
for i in range(n_test-1):
    error += abs((Y_test[i] - Y_pred[i])/Y_test[i])
error = error/n_test

print("MEAN ABSOLUTE PERCENTAGE ERROR: ")
float(error*100)

MEAN ABSOLUTE PERCENTAGE ERROR: 


23.185650124145248